In [55]:
from tkinter import Canvas, Label, Tk

In [56]:
# reload module
%load_ext autoreload
%autoreload 2

from calculations import Calculations
from point import Point
from cursor_manager import CursorManager
from mixins.mix_event_bindings import mix_event_bindings
from history import History

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
class CurveGrid(Canvas, Calculations, Point):

    def __init__(self, master=None, **kwargs):
        super().__init__(master, **kwargs)
        Point.__init__(self)

        # classes
        self.cursor = CursorManager(self)
        ###self.buffer = History()

        # events
        self.root = master
        mix_event_bindings(self, self.root)

        # create Label coords
        self.label = Label( root, text=f"x: 0\ny: 0", anchor="w", justify="left", bg="#333333", fg="#aaaaaa", font=("Arial", 14))
        self.label.place(x=50, y=580)

        # init grid & line
        self.draw_grid()
        self.draw_line()

        ###self.buffer.save_step(self.points)
        ###self.buffer.show_history()

    def draw_grid(self):
        # vertical lines
        for coord in range(50, 561, 102):
            self.create_line(coord, 50, coord, 561, fill="#444444")
            self.create_line(50, coord, 561, coord, fill="#444444")


    def draw_line(self):
        self.delete("curve", "point")

        # curve coords
        self.curve_points = self.curve_coords(self.points[1:-1])

        # curve
        self.create_line(
            # left fix point, curve points, right fix point
            self.points[0], *self.curve_points, self.points[-1],
            fill="#ffffbb",
            width=1,
            tags="curve"
        )

        # points
        for i, (x, y) in enumerate(self.points, start=0):
            if i not in (0, len(self.points)-1):
                self.draw_point(x, y)

    def on_enter(self, event):
        if self.allow_add_point(event):
            if self.allow_influence_point(event):
                self.cursor.hand_cursor(event)
            else:
                self.cursor.cross_cursor()
        else:
            if self.allow_influence_point(event):
                if not event.state & 0x4:
                    self.cursor.hand_cursor()

            else:
                self.cursor.arrow_cursor()

    def left_click(self, event):
        self.x, self.y = event.x, event.y

        # ctrl + LMB
        if event.state & 0x4:
            if self.allow_influence_point(event):
                self.remove_point()
                self.draw_line()
        else:
            # add a point to the curve & redraw the curve
            if self.allow_add_point(event):
                self.add_point()
                self.draw_line()

            # choose the nearest point
            self.choose_point(event)

    # drag point
    def on_drag(self, event):
        self.configure(cursor=self.cursor.drag_path)

        # restricted movement coordinates
        self.x = max(self.start_x, min(event.x, self.end_x))
        self.y = max(self.end_y, min(event.y, self.start_y))

        # selected point
        self.points[self.selected_index] = (self.x, self.y)

        # coord: x -> next point, previous point
        prev_x = self.points[self.selected_index - 1][0]
        next_x = self.points[self.selected_index + 1][0]

        # limit min distance between points
        if self.selected_index != 1 and self.x - prev_x < self.lim['x']:
            self.limited_point(prev_x + self.lim['x'])
        elif self.selected_index != len(self.points)-2 and next_x - self.x < self.lim['x']:
            self.limited_point(next_x - self.lim['x'])

        self.label.config(text=f"x: {round((self.x - 50) / 2)}\ny: {round((560 - self.y) / 2)}")

        # boundary invisible points
        if self.selected_index == 1:
            self.points[0] = (self.start_x, self.y)
        elif self.selected_index == len(self.points) - 2:
            self.points[-1] = (self.end_x, self.y)

        self.draw_line()

    def on_release(self, event):
        self.selected_index = None
        self.on_enter(event)


In [58]:
root = Tk()
root.focus_force()

canvas = CurveGrid(root, width=600, height=650, bg="#333333")
canvas.pack(expand=True, fill="both")
canvas.configure(bg="#333333")

root.title("сurves")
root.mainloop()

In [59]:
# points = [(50, 560), (150, 200), (250, 100), (560, 50)]